# Transfer Learning with Transformers using Hugging Face

© Data Trainers LLC. GPL v 3.0.

**Author**: Axel Sirota

In this notebook, we will explore the process of transfer learning using transformers, specifically leveraging the Hugging Face library. Hugging Face offers a range of pre-trained models that allow you to quickly implement state-of-the-art NLP solutions without the need to train models from scratch. We will guide you through the steps to prepare, fine-tune, and evaluate a transformer model for a specific NLP task.

### Introduction to Hugging Face

Hugging Face is an open-source company that provides a wide array of transformer models and tools for NLP tasks. With their easy-to-use API and extensive model repository, you can implement advanced machine learning models with minimal effort. In this notebook, we'll focus on using these resources to apply transfer learning, which involves taking a pre-trained model and fine-tuning it on a new dataset to solve a different task.

## Preparation

Before we dive into the code, let's make sure we have all the necessary libraries installed. We'll be using the `datasets`, `evaluate`, and `transformers` libraries from Hugging Face, along with `sentencepiece` for handling various NLP tasks.

In [ ]:
# Install required libraries
!pip install -U datasets evaluate transformers transformers[sentencepiece]

### Importing Libraries

Now that we have the required packages installed, let's import them into our environment. We will also set some global parameters such as `EPOCHS` and `BATCH_SIZE`, and suppress any unnecessary warnings.

In [ ]:
# Import necessary libraries for the task
import tensorflow as tf  # TensorFlow for deep learning
from datasets import load_dataset  # Hugging Face's library to load datasets
from transformers import AutoTokenizer  # Tokenizer from Hugging Face's transformers library
import numpy as np  # NumPy for numerical operations
import random  # Random module for reproducibility
import pandas as pd  # Pandas for data manipulation
import warnings  # Warnings to control unnecessary alerts

# Define global parameters for training
EPOCHS = 3  # Number of training epochs
BATCH_SIZE = 64  # Batch size for training
# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

## Tokenizing and loading the dataset

In HuggingFace there are many models, and each has its own tokenizer. Lucky for us there is a class `AutoTokenizer` that does the heavylifting after we provide a checkpoint

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

raw_datasets = load_dataset("imdb")  # load imdb dataset
raw_datasets

Notice it is a dict object with the train, test, and unsupervised datasets to play around

In [ ]:
raw_datasets['train'][0]  # Let's see the first review

How do we know if it's positive or negative from label=0?

In [ ]:
raw_datasets['train'].features

There it is, within features we see that the index 0 is **Negative**

Now to tokenise the dataset we need to load the proper tokenizer for the model we care about. And the we are goin to apply it everywhere!

After this step the tokenizer converts the text into a Tensor of ids, each representing a diferent word in the BERT vocabulary

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = None # Fetch the tokenizer for that checkpoint


def tokenize_function(example):
    # We are using the BERT tokenizer, specifying to PAD until the end,
    # truncate if either 128 elements are met or the maximum from the model, which you get from the model card

    return pass # Return a tokenizer function that adds padding to 128 chars and truncates from the examples


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)


Let's see how it worked!

In [ ]:
tokenized_datasets['train'][0]['text']

In [ ]:
tokenizer(tokenized_datasets['train'][0]['text'])

The tokenizer from BERT (well DistillBERT) converts each word into its ID according to *its* vocabulary. And notice the masking says we haven't been truncated. What we will do know is do this for all data and convert it into a TF Datasets object (which Keras accepts)

In [ ]:

tf_train_dataset = None # Convert the tokenized_datasets["train"] to a TF Dataset


tf_validation_dataset = None. # Same with validation

In [ ]:
for inputs, labels in tf_train_dataset.take(1):
  print(f' inputs: {inputs.shape}, labels: {labels.shape}')


## Downloading the model and prepare for training

Now let's download the model. It is very important you use the class that starts with `TFAutoModel`. There are auto models for most tasks, so you don't have to manually add the header, for example the `TFAutoModelForSequenceClassification` adds a Dense layer (WITHOUT SOFTMAX) to do the classification

In [ ]:

model = None # Download the model for sequence classification with 2 labels (sentiment analysis)

In [ ]:
loss = None  # Set the loss
# Compile the model

In [ ]:
model.summary()

Oh no! We have too many parameters to train! Luckily in Keras is very easy to set some layers as not trainable

In [ ]:
# Set the first layer as non trainable

In [ ]:
model.summary()

*Voilá!*

In [ ]:
# Fit the model

Now we have a trained model that did transfer learning from DistillBERT

## Testing it out!

In [ ]:
tokens = tokenizer(["This is the worst internet service provider", "Although most people say this is the worst, I like it"], padding=True, truncation=True, max_length=128)

In [ ]:
tokens

In [ ]:
model.predict(tokens['input_ids'])

Notice the prediction where not probabilities but logits!

In [ ]:
tf.math.softmax(model.predict(tokens['input_ids'])['logits'])

In [ ]:
tf.math.argmax(tf.math.softmax(model.predict(tokens['input_ids'])['logits']))

And the model was correct!!

In [ ]:
model.evaluate(tf_validation_dataset)